# Modeling Component Area, Energy, Latency, and Leak Power

This notebook shows how to integrate component models from the `hwcomponents` library to
model area, energy, latency, and leak power.

Before reading this notebook, you should be familiar with the `hwcomponents` library.
Ensure you've installed it and been through all of the notebooks in the [hwcomponents
documentation](https://accelergy-project.github.io/hwcomponents/).

We'll be using the `component_example.yaml` file, which is annotated explaining what
each attribute means.

In [ ]:
from pathlib import Path
from IPython.display import Markdown

examples_dir = Path("../../examples")

display(Markdown(f"""
```yaml
{open(examples_dir / "misc" / "component_annotated.yaml").read()}
```
"""))

Now we'll create a global buffer component and model its area, energy, latency, and leak
power.

In [ ]:
# < DOC_INCLUDE_MARKER > single_component_energy_area

# Create the component from the yaml file
import accelforge as af
global_buffer = af.arch.Memory.from_yaml(
    examples_dir / "misc" / "component_annotated.yaml"
)

# Calculate the energy and area of the component
global_buffer = global_buffer.calculate_area_energy_latency_leak()

# Print out the area, leak power, and energy for each action
area = global_buffer.attributes.area
leak_power = global_buffer.attributes.leak_power
print(f"Area: {area:.2e} m^2")
print(f"Leak power: {leak_power:.2e} W")
for action in global_buffer.actions:
    energy = action.arguments.energy
    latency = action.arguments.latency
    print(f'Action {action.name} energy {energy:.2e} J, latency {latency:.2e}s')

Calculating area, energy, latency, and leak power for a component fills out more
attributes as well! We can inspect what is available by looking at the __doc__ or by
looking at this project's documentation website.

In [ ]:
print(global_buffer.calculate_area_energy_latency_leak.__doc__)

As we can see in the doc, this function also fills out the `component_modeling_log`
attribute of the component, which contains messages from the component modeling
calculations.

In [ ]:
# < DOC_INCLUDE_MARKER > hwcomponents
for message in global_buffer.component_modeling_log:
    print(message)

To get more details on the specific model used, we can also look at the
`component_model` attribute of the component, which has the
`hwcomponents.ComponentModel` object that was used to calculate the energy and area. We
can use `help()` on this object to get more details on the model.

In [ ]:
# < DOC_INCLUDE_MARKER > hwcomponents
component_model = global_buffer.component_model
print(help(component_model)) # Get documentation for the model

We can see that the global buffer is an instance of
`hwcomponents_library.SmartBufferSRAM`, and it contains four subcomponents: an SRAM
buffer, an address register, a delta register, and an addder. We can look at each of
these components to see the area breakdown of our buffer.

In [ ]:
# < DOC_INCLUDE_MARKER > hwcomponents

# The model is a hwcomponents_library.SmartBufferSRAM object!
from hwcomponents_library import SmartBufferSRAM

# Type hint it for our IDE
component_model: SmartBufferSRAM = global_buffer.component_model

# Inspect the area of each of its subcomponents
print(f'Area: {component_model.area:.2e} m^2')
print(f'\tSRAM area: {component_model.sram.area:.2e} m^2')
print(f'\tAdder area: {component_model.adder.area:.2e} m^2')
print(f'\tAddress register area: {component_model.address_reg.area:.2e} m^2')
print(f'\tDelta reg area: {component_model.delta_reg.area:.2e} m^2')

Now we'll model the components for a larger design using the `eyeriss_like`
architecture.

In [ ]:
display(Markdown(f"""
```yaml
{open(examples_dir / "arches" / "eyeriss_like.arch.yaml").read()}
```
"""))

The top-level specification object has a `calculate_area_energy_latency_leak` method
that can be used to calculate the area, energy, latency, and leak power of all
components in the design. Additionally, when called from the top-level specification,
this method parses expressions in the architecture, letting you express attributes as
functions of other attributes.

In [ ]:
# < DOC_INCLUDE_MARKER > spec_energy_area

# Load the spec and calculate the energy and area of the entire design
spec = af.Spec.from_yaml(examples_dir / "arches" / "eyeriss_like.arch.yaml")
spec = spec.calculate_component_area_energy_latency_leak()

# Print out the total area and leakage power of the entire design
print(f'Total area of the design: {spec.arch.total_area:.2e} m^2')
print(f'Area breakdown per component:')
for component, area in spec.arch.per_component_total_area.items():
    print(f'\t{component}: {area:.2e} m^2')
print(f'Total leakage power of the design: {spec.arch.total_leak_power:.2e} W')
for component, leak_power in spec.arch.per_component_total_leak_power.items():
    print(f'\t{component}: {leak_power:.2e} W')

Once the area, energy, latency, and leak power of the architecture are calculated, we can
look at any individual component and analyze it like we analyzed the global buffer.

In [ ]:
mac = spec.arch.find("MAC")
print(f'MAC area: {mac.attributes.area:.2e} m^2')
print(f'Total area of all MACs in the architecture: {mac.attributes.total_area:.2e} m^2')
print(f'MAC leak power: {mac.attributes.leak_power:.2e} W')
print(f'Total leak power of all MACs in the architecture: {mac.attributes.total_leak_power:.2e} W')
for action in mac.actions:
    energy = action.arguments.energy
    latency = action.arguments.latency
    print(f'{action.name} energy: {energy:.2e} J, latency {latency:.2e}s')

We can also force the design to use a specific model for a component by passing in our
own `ComponentModel` object. Here we'll create a custom MAC model that has lower
leakage power than the default model.












In [ ]:
# Create a hwcomponents model. See the hwcomponents tutorial for ore information on how
# to make these!
import hwcomponents as hwc

class MyMac(hwc.ComponentModel):
    component_name: str = "intmac"
    priority: float = 0.5

    def __init__(self):
        super().__init__(
            area=5e-8,
            leak_power=1e-12 # Very low leakage power
        )

    @hwc.action
    def compute(self) -> float:
        self.logger.info(f'*** Getting compute energy ***')
        # 1pJ, 1ns
        return 1e-12, 1e-9

# Initialize the spec and make the MAC use our custom model
spec = af.Spec.from_yaml(examples_dir / "arches" / "eyeriss_like.arch.yaml")
mac = spec.arch.find("MAC")
mac.component_model = MyMac()

# Calculate the energy and area of the MAC and print out the results
mac = mac.calculate_area_energy_latency_leak()
print(f'MAC area: {mac.attributes.area:.2e} m^2')
print(f'MAC leak power: {mac.attributes.leak_power:.2e} W')
for action in mac.actions:
    energy = action.arguments.energy
    latency = action.arguments.latency
    print(f'{action.name} energy: {energy:.2e} J, latency {latency:.2e}s')

# The energy and area log contain logs from the ComponentModel component.
print(f"Log messages: ")
for message in mac.component_modeling_log:
    print(f"\t{message}")

The `hwcomponents` library has plenty of other component models to use! These can be
listed with the `hwcomponents.get_models` function.

If you're interested in a specific model, use the `help()` function to get more
information on it.

For more information, see the [hwcomponents
documentation](https://accelergy-project.github.io/hwcomponents/).

In [ ]:
import hwcomponents as hwc

for model in hwc.get_models()[:5]:
    print(f"{model} supports {model.component_name}")
    for action in model.get_action_names():
        print(f'\t{action}')


from hwcomponents_adc import ADC
help(ADC)